<a href="https://www.kaggle.com/code/nkirichimzifrancis/assignment-2-p2838878-nkiri-chimzi-francis?scriptVersionId=239324878" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install pyspark==3.2

In [ ]:
# start spark session and configureation
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.master("local[2]").appName("dat").getOrCreate()
sc = spark.sparkContext
#create an instance of SQLContext
sqlContext = SQLContext(spark)

In [ ]:
from io import StringIO
from collections import namedtuple
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

import csv
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

In [ ]:
UKCrimereport = spark.read.option("header", "true").option("delimiter", ",")\
.option("inferSchema", "true").\
    csv("/kaggle/input/citycrimes/CITIES/*street.csv")

In [ ]:
UKCrimereport.count()

In [ ]:
UKCrimereport.printSchema()

In [ ]:
UKCrimereport.show()

In [ ]:
New_UKCrimereport = UKCrimereport.drop('Context')

In [ ]:
New_UKCrimereport.printSchema()

In [ ]:
New_UKCrimereport = New_UKCrimereport.withColumnRenamed('Crime ID', 'Crime_ID')
New_UKCrimereport = New_UKCrimereport.withColumnRenamed('Reported by', 'Reported_by')
New_UKCrimereport = New_UKCrimereport.withColumnRenamed('Falls within', 'Falls_within')
New_UKCrimereport = New_UKCrimereport.withColumnRenamed('LSOA code', 'LSOA_code')
New_UKCrimereport = New_UKCrimereport.withColumnRenamed('LSOA name', 'LSOA_name')
New_UKCrimereport = New_UKCrimereport.withColumnRenamed('Crime type', 'Crime_type')
New_UKCrimereport = New_UKCrimereport.withColumnRenamed('Last outcome category', 'Last_outcome_category')

In [ ]:
New_UKCrimereport.printSchema()

In [ ]:
#register temp tables

New_UKCrimereport.registerTempTable("tbl_UKCrimereport")

In [ ]:
sqlContext.sql("select * from tbl_UKCrimereport").show(5)

**ANALYSIS ON LEICESTERSHIRE CRIME DATA**

In [ ]:
#Only reports from Leicestershire police

Leicestershire_Crime_Count = New_UKCrimereport.filter(col("Reported_by") == "Leicestershire Police")

In [ ]:
Leicestershire_Crime_Count.count()

In [ ]:
#Leicestershire report

Leicestershire_Crime_Count.show()

In [ ]:
# A) Analysis Monthly Trends In Criminal Activity

#Leicestershire Crime amount by month 

Leicestershire_Crime_CountNew = Leicestershire_Crime_Count.groupby("Month").count().orderBy(desc("count"))

In [ ]:
#Result for Leicestershire Crime amount by month

Leicestershire_Crime_CountNew.show()

In [ ]:
#Extracting both year and month from the Leicestershire Monthly count

Leicestershire_Crime_withYear = Leicestershire_Crime_CountNew.select("Month",
    substring("Month", 1, 4).alias("Year"),# Extract the year from 'Month'
    substring("Month", 6, 2).alias("Months"), # Extract the month from 'Month'
    "count" )  # Include the existing 'count' column

In [ ]:
#Result of both year and month extracted from the Leicestershire Monthly count

Leicestershire_Crime_withYear.show()

In [ ]:
#Crime amount for each month in 2022

Leicestershire_Crime_Count_2022 = Leicestershire_Crime_withYear.filter(Leicestershire_Crime_withYear.
   Year =="2022" ). select("Year","Months", "count").orderBy("Months")

In [ ]:
#Results of Crime amount for each month in 2022

Leicestershire_Crime_Count_2022.show()

In [ ]:
#Crime amount for each month in 2023

Leicestershire_Crime_Count_2023 = Leicestershire_Crime_withYear.filter(Leicestershire_Crime_withYear.
   Year =="2023" ). select("Year","Months", "count").orderBy("Months")

In [ ]:
#Results of Crime amount for each month in 2023

Leicestershire_Crime_Count_2023.show()

In [ ]:
#USING SQL SYNTAX to Analyse Monthly Trends In Criminal Activity 

#Leicestershire Crime amount by month in 2022 only using SQL

sqlContext.sql("SELECT Month, \
        substring(Month, 1, 4) AS Year,  substring(Month, 6, 2) AS MonthOnly, COUNT(*) AS CrimeCount \
    FROM tbl_UKCrimereport \
    WHERE Reported_by = 'Leicestershire Police' AND substring(Month, 1, 4) = '2022' \
    GROUP BY Month \
    ORDER BY MonthOnly").show()

In [ ]:
#Leicestershire Crime amount by month in 2023 only using SQL

sqlContext.sql("SELECT Month, \
        substring(Month, 1, 4) AS Year,  substring(Month, 6, 2) AS MonthOnly, COUNT(*) AS CrimeCount \
    FROM tbl_UKCrimereport \
    WHERE Reported_by = 'Leicestershire Police' AND substring(Month, 1, 4) = '2023' \
    GROUP BY Month \
    ORDER BY MonthOnly").show()

In [ ]:
Leicestershire_Crime_Count_2022.registerTempTable("Leicestershire_2022")
sqlContext.sql("select * from Leicestershire_2022").show()

In [ ]:
# Prepare variables for visualisations for Leicestershire Monthly Trends in 2022

monthArray = sqlContext.sql("select Months from Leicestershire_2022")\
.rdd.map(lambda f:f[0]).collect()
monthlyCount = sqlContext.sql("select count from Leicestershire_2022")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = monthArray
y = monthlyCount

plt.plot(x, y, 'o-')
plt.title('Monthly Crime Count for 2022')
plt.ylabel('Number of Crime')
plt.xlabel('Month')
plt.grid(True)
plt.savefig("Leicestershire2022CrimeAmount.png")
plt.show()

In [ ]:
Leicestershire_Crime_Count_2023.registerTempTable("Leicestershire_2023")
sqlContext.sql("select * from Leicestershire_2023").show()

In [ ]:
# Prepare variables for visualisations for Leicestershire Monthly Trends in 2023

monthArray = sqlContext.sql("select Months from Leicestershire_2023")\
.rdd.map(lambda f:f[0]).collect()
monthlyCount = sqlContext.sql("select count from Leicestershire_2023")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = monthArray
y = monthlyCount

plt.plot(x, y, 'o-')
plt.title('Monthly Crime Count for 2023')
plt.ylabel('Number of Crime')
plt.xlabel('Month')
plt.grid(True)
plt.savefig("Leicestershire2023CrimeAmount.png")
plt.show()

In [ ]:
# B) Analysis by year to observe annual trends

#Leicestershire Crime amount by year sorted descendantly

Leicestershire_Annual_Count = Leicestershire_Crime_Count.select(substring(Leicestershire_Crime_Count.Month, 1,4).alias('Year'), "Month")\
.groupby("Year").count().orderBy(desc("count"))

In [ ]:
Leicestershire_Annual_Count.show()

In [ ]:
# Using SQL Syntax to Analyse by year to observe annual trends

#Leicestershire Crime amount by year sorted descendantly using SQL 

sqlContext.sql("SELECT substring(Month, 1, 4) AS Year, COUNT(*) AS TotalCrimes\
    FROM tbl_UKCrimereport\
    WHERE Reported_by = 'Leicestershire Police'\
    GROUP BY Year\
    ORDER BY TotalCrimes Desc").show()

In [ ]:
Leicestershire_Annual_Count.registerTempTable("tbl_Leicestershire_Annual_Count")
sqlContext.sql("select * from tbl_Leicestershire_Annual_Count").show()

In [ ]:
# Prepare variables for visualisations for Leicestershire annual trends

YearlyArray = sqlContext.sql("select Year from tbl_Leicestershire_Annual_Count")\
.rdd.map(lambda f:f[0]).collect()
YearlyCount = sqlContext.sql("select count from tbl_Leicestershire_Annual_Count")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = YearlyArray
y = YearlyCount

plt.plot(x, y, 'o-')
plt.title('ANNUAL TRENDS')
plt.ylabel('Crime Frequency')
plt.xlabel('YEAR')
plt.grid(True)
plt.savefig("Leicestershire_Annual_Count.png")
plt.show()

In [ ]:
# C) Top crime types to Identify the most frequent crime categories

#Various crime types for Leicestershire

Leicestershire_CrimeTypes = sqlContext.sql("select Crime_type, Month, \
substring(Month, 0, 4) as Year,\
count(*) as Count \
from tbl_UKCrimereport \
WHERE Reported_by = 'Leicestershire Police'\
Group By Month, Crime_type")

In [ ]:
Leicestershire_CrimeTypes.select("Crime_type").distinct().count()

In [ ]:
Leicestershire_CrimeTypes.show()

In [ ]:
# Filter for only records for 2022

Leicestershire_CrimeType_2022 = Leicestershire_CrimeTypes.filter(Leicestershire_CrimeTypes["Year"] == "2022")

In [ ]:
#Show result for 2022

Leicestershire_CrimeType_2022.show()

In [ ]:
# Group by Leicestershire Crime_type and count occurrences for 2022

Leicestershire_CrimeType_for_2022 = Leicestershire_CrimeType_2022.groupBy("Crime_type").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
#Show grouped result for 2022

Leicestershire_CrimeType_for_2022.show()

In [ ]:
# Filter for only records for 2023

Leicestershire_CrimeType_2023  = Leicestershire_CrimeTypes.filter(Leicestershire_CrimeTypes["Year"] == "2023")

In [ ]:
#Show result for 2023

Leicestershire_CrimeType_2023.show()

In [ ]:
# Group by Leicestershire Crime_type and count occurrences for 2023

Leicestershire_CrimeType_for_2023 = Leicestershire_CrimeType_2023.groupBy("Crime_type").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
#Show grouped result for 2023

Leicestershire_CrimeType_for_2023.show()

In [ ]:
#Registering temp table for Leicestershire Crime Types

Leicestershire_CrimeTypes.registerTempTable("tbl_Leicestershire_CrimeTypes")

In [ ]:
#Using SQL Syntax to Filter for only 2022 Leicestershire Crime Type

Leicestershire_CrimeTypes_2022 = sqlContext.sql("SELECT Crime_type, \
    SUM(Count) AS Count \
    FROM tbl_Leicestershire_CrimeTypes \
    WHERE substring(Month, 1, 4) = '2022' \
    GROUP BY Crime_type \
    Order by Count Desc")

In [ ]:
#Result of  2022 Leicestershire Crime Type using SQL

Leicestershire_CrimeTypes_2022.show()

In [ ]:
#Using SQL Syntax to Filter for only 2023 Leicestershire Crime Type

Leicestershire_CrimeTypes_2023 = sqlContext.sql("SELECT Crime_type, \
    SUM(Count) AS Count \
    FROM tbl_Leicestershire_CrimeTypes \
    WHERE substring(Month, 1, 4) = '2023' \
    GROUP BY Crime_type \
    Order by Count Desc")

In [ ]:
#Result of  2023 Leicestershire Crime Type using SQL

Leicestershire_CrimeTypes_2023.show()

In [ ]:
#Creating temp table for visualisation of  Leicestershire Crime Type of 2022

Leicestershire_CrimeType_for_2022.registerTempTable("Leicestershire_2022CrimeType")
sqlContext.sql("select * from Leicestershire_2022CrimeType").show()

In [ ]:
#Creating temp table for visualisation of  Leicestershire Crime Type of 2023

Leicestershire_CrimeType_for_2023.registerTempTable("Leicestershire_2023CrimeType")
sqlContext.sql("select * from Leicestershire_2023CrimeType").show()

In [ ]:
# Prepare variables for visualisations for Leicestershire Crime Type 2022

Crimetype = sqlContext.sql("select Crime_type from Leicestershire_2022CrimeType")\
.rdd.map(lambda f:f[0]).collect()
CrimeCount = sqlContext.sql("select Total_Count from Leicestershire_2022CrimeType")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = Crimetype
y = CrimeCount

plt.barh(x, y, color='skyblue')
plt.title('Crime Type Trend 2022')
plt.ylabel('Crime Type')
plt.xlabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.savefig("Leicestershire_2022CrimeType.png")
plt.show()

In [ ]:
# Prepare variables for visualisations for Leicestershire Crime Type 2023

Crimetype = sqlContext.sql("select Crime_type from Leicestershire_2023CrimeType")\
.rdd.map(lambda f:f[0]).collect()
CrimeCount = sqlContext.sql("select Total_Count from Leicestershire_2023CrimeType")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = Crimetype
y = CrimeCount

plt.barh(x, y, color='skyblue')
plt.title('Crime Type Trend 2023')
plt.ylabel('Crime Type')
plt.xlabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.savefig("Leicestershire_2023CrimeType.png")
plt.show()

In [ ]:
# D) Location Analysis

Leicestershire_CrimeLocations = sqlContext.sql("select Location, Month, \
substring(Month, 0, 4) as Year,\
count(*) as Count \
from tbl_UKCrimereport \
WHERE Reported_by = 'Leicestershire Police'\
Group By Month, Location")

In [ ]:
Leicestershire_CrimeLocations.select("Location").distinct().count()

In [ ]:
Leicestershire_CrimeLocations.show()

In [ ]:
# Filter for only records for 2022

Leicestershire_CrimeLocations_2022 = Leicestershire_CrimeLocations.filter(Leicestershire_CrimeLocations["Year"] == "2022")

In [ ]:
Leicestershire_CrimeLocations_2022.show()

In [ ]:
# Group by Leicestershire Crime Location and count occurrences for 2022

Leicestershire_CrimeLocations_for_2022 = Leicestershire_CrimeLocations_2022.groupBy("Location").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
Leicestershire_CrimeLocations_for_2022.show()

In [ ]:
# Filter for only records for 2023

Leicestershire_CrimeLocations_2023 = Leicestershire_CrimeLocations.filter(Leicestershire_CrimeLocations["Year"] == "2023")

In [ ]:
# Group by Leicestershire Crime Location and count occurrences for 2023

Leicestershire_CrimeLocations_for_2023 = Leicestershire_CrimeLocations_2023.groupBy("Location").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
Leicestershire_CrimeLocations_for_2023.show(truncate=False)

**ANALYSIS ON NOTTINGHAMSHIRE CRIME DATA**

In [ ]:
#Only reports from Nottinghamshire police

Nottinghamshire_Crime_Count = New_UKCrimereport.filter(col("Reported_by") == "Nottinghamshire Police")

In [ ]:
Nottinghamshire_Crime_Count.count()

In [ ]:
Nottinghamshire_Crime_Count.show()

In [ ]:
# A) Analysis Monthly Trends In Criminal Activity

#Nottinghamshire Crime amount by month 

Nottinghamshire_Crime_CountNew = Nottinghamshire_Crime_Count.groupby("Month").count().orderBy(desc("count"))

In [ ]:
#Result for Nottinghamshire Crime amount by month

Nottinghamshire_Crime_CountNew.show()

In [ ]:
#Extracting both year and month from the Nottinghamshire Monthly count

Nottinghamshire_Crime_withYear = Nottinghamshire_Crime_CountNew.select("Month",
    substring("Month", 1, 4).alias("Year"),# Extract the year from 'Month'
    substring("Month", 6, 2).alias("Months"), # Extract the month from 'Month'
    "count" )  # Include the existing 'count' column

In [ ]:
#Result of both year and month extracted from the Nottinghamshire Monthly count

Nottinghamshire_Crime_withYear.show()

In [ ]:
#Crime amount for each month in 2022

Nottinghamshire_Crime_Count_2022 = Nottinghamshire_Crime_withYear.filter(Nottinghamshire_Crime_withYear.
   Year =="2022" ). select("Year","Months", "count").orderBy("Months")

In [ ]:
#Results of Crime amount for each month in 2022

Nottinghamshire_Crime_Count_2022.show()

In [ ]:
#Crime amount for each month in 2023

Nottinghamshire_Crime_Count_2023 = Nottinghamshire_Crime_withYear.filter(Nottinghamshire_Crime_withYear.
   Year =="2023" ). select("Year","Months", "count").orderBy("Months")

In [ ]:
#Results of Crime amount for each month in 2023

Nottinghamshire_Crime_Count_2023.show()

In [ ]:
Nottinghamshire_Crime_Count_2022.registerTempTable("Nottinghamshire_2022")
sqlContext.sql("select * from Nottinghamshire_2022").show()

In [ ]:
# Prepare variables for visualisations for Nottinghamshire Monthly Trends in 2022

monthArray = sqlContext.sql("select Months from Nottinghamshire_2022")\
.rdd.map(lambda f:f[0]).collect()
monthlyCount = sqlContext.sql("select count from Nottinghamshire_2022")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = monthArray
y = monthlyCount

plt.plot(x, y, 'o-')
plt.title('Monthly Crime Count for 2022')
plt.ylabel('Number of Crime')
plt.xlabel('Month')
plt.grid(True)
plt.savefig("Nottinghamshire2022CrimeAmount.png")
plt.show()

In [ ]:
Nottinghamshire_Crime_Count_2023.registerTempTable("Nottinghamshire_2023")
sqlContext.sql("select * from Nottinghamshire_2023").show()

In [ ]:
# Prepare variables for visualisations for Nottinghamshire Monthly Trends in 2022

monthArray = sqlContext.sql("select Months from Nottinghamshire_2023")\
.rdd.map(lambda f:f[0]).collect()
monthlyCount = sqlContext.sql("select count from Nottinghamshire_2023")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = monthArray
y = monthlyCount

plt.plot(x, y, 'o-')
plt.title('Monthly Crime Count for 2023')
plt.ylabel('Number of Crime')
plt.xlabel('Month')
plt.grid(True)
plt.savefig("Nottinghamshire2023CrimeAmount.png")
plt.show()

In [ ]:
# B) Analysis by year to observe annual trends

#Nottinghamshire Crime amount by year sorted descendantly

Nottinghamshire_Annual_Count = Nottinghamshire_Crime_Count.select(substring(Nottinghamshire_Crime_Count.Month, 1,4).alias('Year'), "Month")\
.groupby("Year").count().orderBy(desc("count"))

In [ ]:
Nottinghamshire_Annual_Count.show()

In [ ]:
Nottinghamshire_Annual_Count.registerTempTable("tbl_Nottinghamshire_Annual_Count")
sqlContext.sql("select * from tbl_Nottinghamshire_Annual_Count").show()

In [ ]:
# Prepare variables for visualisations for Nottinghamshire annual trends

YearlyArray = sqlContext.sql("select Year from tbl_Nottinghamshire_Annual_Count")\
.rdd.map(lambda f:f[0]).collect()
YearlyCount = sqlContext.sql("select count from tbl_Nottinghamshire_Annual_Count")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = YearlyArray
y = YearlyCount

plt.plot(x, y, 'o-')
plt.title('ANNUAL TRENDS')
plt.ylabel('Crime Frequency')
plt.xlabel('YEAR')
plt.grid(True)
plt.savefig("Nottinghamshire_Annual_Count.png")
plt.show()

In [ ]:
# C) Top crime types to Identify the most frequent crime categories

#Various crime types for Nottinghamshire

Nottinghamshire_CrimeTypes = sqlContext.sql("select Crime_type, Month, \
substring(Month, 0, 4) as Year,\
count(*) as Count \
from tbl_UKCrimereport \
WHERE Reported_by = 'Nottinghamshire Police'\
Group By Month, Crime_type")

In [ ]:
Nottinghamshire_CrimeTypes.select("Crime_type").distinct().count()

In [ ]:
Nottinghamshire_CrimeTypes.show()

In [ ]:
# Filter for only records for 2022

Nottinghamshire_CrimeType_2022 = Nottinghamshire_CrimeTypes.filter(Nottinghamshire_CrimeTypes["Year"] == "2022")

In [ ]:
#Show result for 2022

Nottinghamshire_CrimeType_2022.show()

In [ ]:
# Group by Nottinghamshire Crime_type and count occurrences for 2022

Nottinghamshire_CrimeType_for_2022 = Nottinghamshire_CrimeType_2022.groupBy("Crime_type").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
#Show grouped result for 2022

Nottinghamshire_CrimeType_for_2022.show()

In [ ]:
# Filter for only records for 2023

Nottinghamshire_CrimeType_2023  = Nottinghamshire_CrimeTypes.filter(Nottinghamshire_CrimeTypes["Year"] == "2023")

In [ ]:
#Show result for 2023

Nottinghamshire_CrimeType_2023.show()

In [ ]:
# Group by Leicestershire Crime_type and count occurrences for 2023

Nottinghamshire_CrimeType_for_2023 = Nottinghamshire_CrimeType_2023.groupBy("Crime_type").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
#Creating temp table for visualisation of  Nottinghamshire Crime Type of 2022

Nottinghamshire_CrimeType_for_2022.registerTempTable("Nottinghamshire_2022CrimeType")
sqlContext.sql("select * from Nottinghamshire_2022CrimeType").show()

In [ ]:
#Creating temp table for visualisation of  Nottinghamshire Crime Type of 2023

Nottinghamshire_CrimeType_for_2023.registerTempTable("Nottinghamshire_2023CrimeType")
sqlContext.sql("select * from Nottinghamshire_2023CrimeType").show()

In [ ]:
# Prepare variables for visualisations for Nottinghamshire Crime Type 2022

Crimetype = sqlContext.sql("select Crime_type from Nottinghamshire_2022CrimeType")\
.rdd.map(lambda f:f[0]).collect()
CrimeCount = sqlContext.sql("select Total_Count from Nottinghamshire_2022CrimeType")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = Crimetype
y = CrimeCount

plt.barh(x, y, color='skyblue')
plt.title('Crime Type Trend 2022')
plt.ylabel('Crime Type')
plt.xlabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.savefig("Nottinghamshire_2022CrimeType.png")
plt.show()

In [ ]:
# Prepare variables for visualisations for Nottinghamshire Crime Type 2023

Crimetype = sqlContext.sql("select Crime_type from Nottinghamshire_2023CrimeType")\
.rdd.map(lambda f:f[0]).collect()
CrimeCount = sqlContext.sql("select Total_Count from Nottinghamshire_2023CrimeType")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = Crimetype
y = CrimeCount

plt.barh(x, y, color='skyblue')
plt.title('Crime Type Trend 2023')
plt.ylabel('Crime Type')
plt.xlabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.savefig("Nottinghamshire_2023CrimeType.png")
plt.show()

In [ ]:
# D) Location Analysis

Nottinghamshire_CrimeLocations = sqlContext.sql("select Location, Month, \
substring(Month, 0, 4) as Year,\
count(*) as Count \
from tbl_UKCrimereport \
WHERE Reported_by = 'Nottinghamshire Police'\
Group By Month, Location")

In [ ]:
Nottinghamshire_CrimeLocations.select("Location").distinct().count()

In [ ]:
Nottinghamshire_CrimeLocations.show()

In [ ]:
# Filter for only records for 2022

Nottinghamshire_CrimeLocations_2022 = Nottinghamshire_CrimeLocations.filter(Nottinghamshire_CrimeLocations["Year"] == "2022")

In [ ]:
Nottinghamshire_CrimeLocations_2022.show()

In [ ]:
# Group by Nottinghamshire Crime Location and count occurrences for 2022

Nottinghamshire_CrimeLocations_for_2022 = Nottinghamshire_CrimeLocations_2022.groupBy("Location").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
Nottinghamshire_CrimeLocations_for_2022.show()

In [ ]:
# Filter for only records for 2023

Nottinghamshire_CrimeLocations_2023 = Nottinghamshire_CrimeLocations.filter(Nottinghamshire_CrimeLocations["Year"] == "2023")

In [ ]:
Nottinghamshire_CrimeLocations_2023.show()

In [ ]:
# Group by Nottinghamshire Crime Location and count occurrences for 2023

Nottinghamshire_CrimeLocations_for_2023 = Nottinghamshire_CrimeLocations_2023.groupBy("Location").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
Nottinghamshire_CrimeLocations_for_2023.show()

**ANALYSIS ON LINCOLNSHIRE CRIME DATA**

In [ ]:
#Only reports from Lincolnshire police

Lincolnshire_Crime_Count = New_UKCrimereport.filter(col("Reported_by") == "Lincolnshire Police")

In [ ]:
Lincolnshire_Crime_Count.count()

In [ ]:
Lincolnshire_Crime_Count.show()

In [ ]:
# A) Analysis Monthly Trends In Criminal Activity

#Lincolnshire Crime amount by month 

Lincolnshire_Crime_CountNew = Lincolnshire_Crime_Count.groupby("Month").count().orderBy(desc("count"))

In [ ]:
#Result for #Result for Lincolnshire Crime amount by month

Lincolnshire_Crime_CountNew.show()

In [ ]:
#Extracting both year and month from the Lincolnshire Monthly count

Lincolnshire_Crime_withYear = Lincolnshire_Crime_CountNew.select("Month",
    substring("Month", 1, 4).alias("Year"),# Extract the year from 'Month'
    substring("Month", 6, 2).alias("Months"), # Extract the month from 'Month'
    "count" )  # Include the existing 'count' column

In [ ]:
#Result of both year and month extracted from the Lincolnshire Monthly count

Lincolnshire_Crime_withYear.show()

In [ ]:
#Crime amount for each month in 2022

Lincolnshire_Crime_Count_2022 = Lincolnshire_Crime_withYear.filter(Lincolnshire_Crime_withYear.
   Year =="2022" ). select("Year","Months", "count").orderBy("Months")

In [ ]:
#Results of Crime amount for each month in 2022

Lincolnshire_Crime_Count_2022.show()

In [ ]:
#Crime amount for each month in 2023

Lincolnshire_Crime_Count_2023 = Lincolnshire_Crime_withYear.filter(Lincolnshire_Crime_withYear.
   Year =="2023" ). select("Year","Months", "count").orderBy("Months")

In [ ]:
#Results of Crime amount for each month in 2023

Lincolnshire_Crime_Count_2023.show()

In [ ]:
Lincolnshire_Crime_Count_2022.registerTempTable("Lincolnshire_2022")
sqlContext.sql("select * from Lincolnshire_2022").show()

In [ ]:
# Prepare variables for visualisations for Lincolnshire Monthly Trends in 2022

monthArray = sqlContext.sql("select Months from Lincolnshire_2022")\
.rdd.map(lambda f:f[0]).collect()
monthlyCount = sqlContext.sql("select count from Lincolnshire_2022")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = monthArray
y = monthlyCount

plt.plot(x, y, 'o-')
plt.title('Monthly Crime Count for 2022')
plt.ylabel('Number of Crime')
plt.xlabel('Month')
plt.grid(True)
plt.savefig("Lincolnshire2022CrimeAmount.png")
plt.show()

In [ ]:
Lincolnshire_Crime_Count_2023.registerTempTable("Lincolnshire_2023")
sqlContext.sql("select * from Lincolnshire_2023").show()

In [ ]:
# Prepare variables for visualisations for Lincolnshire Monthly Trends in 2023

monthArray = sqlContext.sql("select Months from Lincolnshire_2023")\
.rdd.map(lambda f:f[0]).collect()
monthlyCount = sqlContext.sql("select count from Lincolnshire_2023")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = monthArray
y = monthlyCount

plt.plot(x, y, 'o-')
plt.title('Monthly Crime Count for 2023')
plt.ylabel('Number of Crime')
plt.xlabel('Month')
plt.grid(True)
plt.savefig("Lincolnshire2023CrimeAmount.png")
plt.show()

In [ ]:
# B) Analysis by year to observe annual trends

#Lincolnshire Crime amount by year sorted descendantly

Lincolnshire_Annual_Count = Lincolnshire_Crime_Count.select(substring(Lincolnshire_Crime_Count.Month, 1,4).alias('Year'), "Month")\
.groupby("Year").count().orderBy(desc("count"))

In [ ]:
Lincolnshire_Annual_Count.show()

In [ ]:
Lincolnshire_Annual_Count.registerTempTable("tbl_Lincolnshire_Annual_Count")
sqlContext.sql("select * from tbl_Lincolnshire_Annual_Count").show()

In [ ]:
# Prepare variables for visualisations for Lincolnshire annual trends

YearlyArray = sqlContext.sql("select Year from tbl_Lincolnshire_Annual_Count")\
.rdd.map(lambda f:f[0]).collect()
YearlyCount = sqlContext.sql("select count from tbl_Lincolnshire_Annual_Count")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = YearlyArray
y = YearlyCount

plt.plot(x, y, 'o-')
plt.title('ANNUAL TRENDS')
plt.ylabel('Crime Frequency')
plt.xlabel('YEAR')
plt.grid(True)
plt.savefig("Lincolnshire_Annual_Count.png")
plt.show()

In [ ]:
# C) Top crime types to Identify the most frequent crime categories

#Various crime types for Lincolnshire

Lincolnshire_CrimeTypes = sqlContext.sql("select Crime_type, Month, \
substring(Month, 0, 4) as Year,\
count(*) as Count \
from tbl_UKCrimereport \
WHERE Reported_by = 'Lincolnshire Police'\
Group By Month, Crime_type")

In [ ]:
Lincolnshire_CrimeTypes.select("Crime_type").distinct().count()

In [ ]:
Lincolnshire_CrimeTypes.show()

In [ ]:
# Filter for only records for 2022

Lincolnshire_CrimeType_2022 = Lincolnshire_CrimeTypes.filter(Lincolnshire_CrimeTypes["Year"] == "2022")

In [ ]:
Lincolnshire_CrimeType_2022.show()

In [ ]:
# Group by London Crime_type and count occurrences for 2022

Lincolnshire_CrimeType_for_2022 = Lincolnshire_CrimeType_2022.groupBy("Crime_type").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
Lincolnshire_CrimeType_for_2022.show()

In [ ]:
# Filter for only records for 2023

Lincolnshire_CrimeType_2023  = Lincolnshire_CrimeTypes.filter(Lincolnshire_CrimeTypes["Year"] == "2023")

In [ ]:
Lincolnshire_CrimeType_2023.show()

In [ ]:
# Group by Leicestershire Crime_type and count occurrences for 2023

Lincolnshire_CrimeType_for_2023 = Lincolnshire_CrimeType_2023.groupBy("Crime_type").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
Lincolnshire_CrimeType_for_2023.show()

In [ ]:
#Creating temp table for visualisation of  Lincolnshire Crime Type of 2022

Lincolnshire_CrimeType_for_2022.registerTempTable("Lincolnshire_2022CrimeType")
sqlContext.sql("select * from Lincolnshire_2022CrimeType").show()

In [ ]:
#Creating temp table for visualisation of  Lincolnshire Crime Type of 2023

Lincolnshire_CrimeType_for_2023.registerTempTable("Lincolnshire_2023CrimeType")
sqlContext.sql("select * from Lincolnshire_2023CrimeType").show()

In [ ]:
# Prepare variables for visualisations for Lincolnshire Crime Type 2022

Crimetype = sqlContext.sql("select Crime_type from Lincolnshire_2022CrimeType")\
.rdd.map(lambda f:f[0]).collect()
CrimeCount = sqlContext.sql("select Total_Count from Lincolnshire_2022CrimeType")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = Crimetype
y = CrimeCount

plt.barh(x, y, color='skyblue')
plt.title('Crime Type Trend 2022')
plt.ylabel('Crime Type')
plt.xlabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.savefig("Lincolnshire_2022CrimeType.png")
plt.show()

In [ ]:
# Prepare variables for visualisations for Lincolnshire Crime Type 2023

Crimetype = sqlContext.sql("select Crime_type from Lincolnshire_2023CrimeType")\
.rdd.map(lambda f:f[0]).collect()
CrimeCount = sqlContext.sql("select Total_Count from Lincolnshire_2023CrimeType")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = Crimetype
y = CrimeCount

plt.barh(x, y, color='skyblue')
plt.title('Crime Type Trend 2023')
plt.ylabel('Crime Type')
plt.xlabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.savefig("Lincolnshire_2023CrimeType.png")
plt.show()

In [ ]:
# D) Location Analysis

Lincolnshire_CrimeLocations = sqlContext.sql("select Location, Month, \
substring(Month, 0, 4) as Year,\
count(*) as Count \
from tbl_UKCrimereport \
WHERE Reported_by = 'Lincolnshire Police'\
Group By Month, Location")

In [ ]:
Lincolnshire_CrimeLocations.select("Location").distinct().count()

In [ ]:
Lincolnshire_CrimeLocations.show()

In [ ]:
# Filter for only records for 2022

Lincolnshire_CrimeLocations_2022 = Lincolnshire_CrimeLocations.filter(Lincolnshire_CrimeLocations["Year"] == "2022")

In [ ]:
Lincolnshire_CrimeLocations_2022.show()

In [ ]:
# Group by Lincolnshire Crime Location and count occurrences for 2022

Lincolnshire_CrimeLocations_for_2022 = Lincolnshire_CrimeLocations_2022.groupBy("Location").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
Lincolnshire_CrimeLocations_for_2022.show()

In [ ]:
# Filter for only records for 2023

Lincolnshire_CrimeLocations_2023 = Lincolnshire_CrimeLocations.filter(Lincolnshire_CrimeLocations["Year"] == "2023")

In [ ]:
Lincolnshire_CrimeLocations_2023.show()

In [ ]:
# Group by Lincolnshire Crime Location and count occurrences for 2023

Lincolnshire_CrimeLocations_for_2023 = Lincolnshire_CrimeLocations_2023.groupBy("Location").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
Lincolnshire_CrimeLocations_for_2023.show()

# ANALYSIS ON CITY OF LONDON CRIME DATA

In [ ]:
#Only reports from City of London police

London_Crime_Count = New_UKCrimereport.filter(col("Reported_by") == "City of London Police")

In [ ]:
London_Crime_Count.count()

In [ ]:
London_Crime_Count.show()

In [ ]:
# A) Analysis Monthly Trends In Criminal Activity

#London Crime amount by month 

London_Crime_CountNew = London_Crime_Count.groupby("Month").count().orderBy(desc("count"))

In [ ]:
London_Crime_CountNew.show()

In [ ]:
#Extracting both year and month from the London Monthly count

London_Crime_withYear = London_Crime_CountNew.select("Month",
    substring("Month", 1, 4).alias("Year"),# Extract the year from 'Month'
    substring("Month", 6, 2).alias("Months"), # Extract the month from 'Month'
    "count" )  # Include the existing 'count' column

In [ ]:
London_Crime_withYear.show()

In [ ]:
#Crime amount for each month in 2022

London_Crime_Count_2022 = London_Crime_withYear.filter(London_Crime_withYear.
   Year =="2022" ). select("Year","Months", "count").orderBy("Months")

In [ ]:
London_Crime_Count_2022.show()

In [ ]:
#Crime amount for each month in 2023

London_Crime_Count_2023 = London_Crime_withYear.filter(London_Crime_withYear.
   Year =="2023" ). select("Year","Months", "count").orderBy("Months")

In [ ]:
London_Crime_Count_2023.show()

In [ ]:
London_Crime_Count_2022.registerTempTable("London_2022")
sqlContext.sql("select * from London_2022").show()

In [ ]:
# Prepare variables for visualisations for London Monthly Trends in 2022

monthArray = sqlContext.sql("select Months from London_2022")\
.rdd.map(lambda f:f[0]).collect()
monthlyCount = sqlContext.sql("select count from London_2022")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = monthArray
y = monthlyCount

plt.plot(x, y, 'o-')
plt.title('Monthly Crime Count for 2022')
plt.ylabel('Number of Crime')
plt.xlabel('Month')
plt.grid(True)
plt.savefig("London2022CrimeAmount.png")
plt.show()

In [ ]:
London_Crime_Count_2023.registerTempTable("London_2023")
sqlContext.sql("select * from London_2023").show()

In [ ]:
# Prepare variables for visualisations for London Monthly Trends in 2023

monthArray = sqlContext.sql("select Months from London_2023")\
.rdd.map(lambda f:f[0]).collect()
monthlyCount = sqlContext.sql("select count from London_2023")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = monthArray
y = monthlyCount

plt.plot(x, y, 'o-')
plt.title('Monthly Crime Count for 2023')
plt.ylabel('Number of Crime')
plt.xlabel('Month')
plt.grid(True)
plt.savefig("London2023CrimeAmount.png")
plt.show()

In [ ]:
# B) Analysis by year to observe annual trends

#London Crime amount by year sorted descendantly

London_Annual_Count = London_Crime_Count.select(substring(London_Crime_Count.Month, 1,4).alias('Year'), "Month")\
.groupby("Year").count().orderBy(desc("count"))

In [ ]:
London_Annual_Count.show()

In [ ]:
London_Annual_Count.registerTempTable("London_Annual_Count")
sqlContext.sql("select * from London_Annual_Count").show()

In [ ]:
# Prepare variables for visualisations for London annual trends

YearlyArray = sqlContext.sql("select Year from London_Annual_Count")\
.rdd.map(lambda f:f[0]).collect()
YearlyCount = sqlContext.sql("select count from London_Annual_Count")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = YearlyArray
y = YearlyCount

plt.plot(x, y, 'o-')
plt.title('ANNUAL TRENDS')
plt.ylabel('Crime Frequency')
plt.xlabel('YEAR')
plt.grid(True)
plt.savefig("London_Annual_Count.png")
plt.show()

In [ ]:
# C) Top crime types to Identify the most frequent crime categories

#Various crime types for London

London_CrimeTypes = sqlContext.sql("select Crime_type, Month, \
substring(Month, 0, 4) as Year,\
count(*) as Count \
from tbl_UKCrimereport \
WHERE Reported_by = 'City of London Police'\
Group By Month, Crime_type")

In [ ]:
London_CrimeTypes.select("Crime_type").distinct().count()

In [ ]:
London_CrimeTypes.show()

In [ ]:
# Filter for only records for 2022

London_CrimeType_2022 = London_CrimeTypes.filter(London_CrimeTypes["Year"] == "2022")

In [ ]:
London_CrimeType_2022.show()

In [ ]:
# Group by London Crime_type and count occurrences for 2022

London_CrimeType_for_2022 = London_CrimeType_2022.groupBy("Crime_type").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
London_CrimeType_for_2022.show()

In [ ]:
# Filter for only records for 2023

London_CrimeType_2023  = London_CrimeTypes.filter(London_CrimeTypes["Year"] == "2023")

In [ ]:
London_CrimeType_2023.show()

In [ ]:
# Group by London Crime_type and count occurrences for 2023

London_CrimeType_for_2023 = London_CrimeType_2023.groupBy("Crime_type").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
London_CrimeType_for_2023.show()

In [ ]:
#Creating temp table for visualisation of  London Crime Type of 2022

London_CrimeType_for_2022.registerTempTable("London_2022CrimeType")
sqlContext.sql("select * from London_2022CrimeType").show()

In [ ]:
#Creating temp table for visualisation of  London Crime Type of 2023

London_CrimeType_for_2023.registerTempTable("London_2023CrimeType")
sqlContext.sql("select * from London_2023CrimeType").show()

In [ ]:
# Prepare variables for visualisations for London Crime Type 2022

Crimetype = sqlContext.sql("select Crime_type from London_2022CrimeType")\
.rdd.map(lambda f:f[0]).collect()
CrimeCount = sqlContext.sql("select Total_Count from London_2022CrimeType")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = Crimetype
y = CrimeCount

plt.barh(x, y, color='skyblue')
plt.title('Crime Type Trend 2022')
plt.ylabel('Crime Type')
plt.xlabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.savefig("London_2022CrimeType.png")
plt.show()

In [ ]:
# Prepare variables for visualisations for London Crime Type 2023

Crimetype = sqlContext.sql("select Crime_type from London_2023CrimeType")\
.rdd.map(lambda f:f[0]).collect()
CrimeCount = sqlContext.sql("select Total_Count from London_2023CrimeType")\
.rdd.map(lambda f:f[0]).collect()


# Plot visualisations

x = Crimetype
y = CrimeCount

plt.barh(x, y, color='skyblue')
plt.title('Crime Type Trend 2023')
plt.ylabel('Crime Type')
plt.xlabel('Frequency')
plt.xticks(rotation=45, ha='right')
plt.savefig("London_2023CrimeType.png")
plt.show()

In [ ]:
# D) Location Analysis

London_CrimeLocations = sqlContext.sql("select Location, Month, \
substring(Month, 0, 4) as Year,\
count(*) as Count \
from tbl_UKCrimereport \
WHERE Reported_by = 'City of London Police'\
Group By Month, Location")

In [ ]:
London_CrimeLocations.show()

In [ ]:
London_CrimeLocations.select("Location").distinct().count()

In [ ]:
# Filter for only records for 2022

London_CrimeLocations_2022 = London_CrimeLocations.filter(London_CrimeLocations["Year"] == "2022")

In [ ]:
London_CrimeLocations_2022.show()

In [ ]:
# Group by London Crime Location and count occurrences for 2022

London_CrimeLocations_for_2022 = London_CrimeLocations_2022.groupBy("Location").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
London_CrimeLocations_for_2022.show()

In [ ]:
# Filter for only records for 2023

London_CrimeLocations_2023 = London_CrimeLocations.filter(London_CrimeLocations["Year"] == "2023")

In [ ]:
London_CrimeLocations_2023.show()

In [ ]:
# Group by London Crime Location and count occurrences for 2023

London_CrimeLocations_for_2023 = London_CrimeLocations_2023.groupBy("Location").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
London_CrimeLocations_for_2023.show()

**COMPARING THE FOUR CITIES**

In [ ]:
# Extract year and month from the date column (assuming column name is 'Month')
Uk_crimedata = New_UKCrimereport.withColumn("Year", year(col("Month"))).withColumn("MonthNum", month(col("Month")))

In [ ]:
Uk_crimedata.show()

In [ ]:
Uk_crime_counts = Uk_crimedata.groupBy("Reported_by", "Year", "MonthNum").agg(count("*").alias("CrimeCount"))

In [ ]:
Uk_crime_counts.show()

In [ ]:
#Compare cities in 2022
Uk_crime_counts_2022 = Uk_crime_counts.filter(col("Year") == 2022)

In [ ]:
Uk_crime_counts_2022.show()

In [ ]:
#Visualisation of the Four counties in year 2022

# Convert PySpark DataFrame to Pandas DataFrame
uk_crime_pd = Uk_crime_counts_2022.toPandas()

# Rename columns for easier handling
uk_crime_pd.rename(columns={'Reported_by': 'County'}, inplace=True)

# Create a pivot table for better visualization
pivot_table = uk_crime_pd.pivot(index='MonthNum', columns='County', values='CrimeCount')

# Plot the data
pivot_table.plot(kind='line', figsize=(15, 6), marker='o')

# Customize the plot
plt.title("Crime Counts by Month for UK Counties (2022)", fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Crime Count", fontsize=12)
plt.legend(title="County", loc='upper right', fontsize=10)
plt.grid(True)
plt.xticks(ticks=range(1, 13), labels=[
    "Jan", "Feb", "Mar", "Apr", "May", "Jun",
    "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"
])
plt.tight_layout()
plt.savefig("Counties_2022Crime_Counts.png")

# Show the plot
plt.show()

In [ ]:
#Compare cities in 2023
Uk_crime_counts_2023 = Uk_crime_counts.filter(col("Year") == 2023)

In [ ]:
Uk_crime_counts_2023.show()

In [ ]:
#Visualisation of the Four counties in year 2023

# Convert PySpark DataFrame to Pandas DataFrame
uk_crime_pd = Uk_crime_counts_2023.toPandas()

# Rename columns for easier handling
uk_crime_pd.rename(columns={'Reported_by': 'County'}, inplace=True)

# Create a pivot table for better visualization
pivot_table = uk_crime_pd.pivot(index='MonthNum', columns='County', values='CrimeCount')

# Plot the data
pivot_table.plot(kind='line', figsize=(15, 6), marker='o')

# Customize the plot
plt.title("Crime Counts by Month for UK Counties (2023)", fontsize=16)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Crime Count", fontsize=12)
plt.legend(title="County", loc='upper right', fontsize=10)
plt.grid(True)
plt.xticks(ticks=range(1, 13), labels=[
    "Jan", "Feb", "Mar", "Apr", "May", "Jun",
    "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"
])
plt.tight_layout()
plt.savefig("Counties_2023Crime_Counts.png")

# Show the plot
plt.show()

In [ ]:
#register temp tables

New_UKCrimereport.registerTempTable("tbl_UKCrimereport")

In [ ]:
# Explore Crime Types per year/month

CrimeTypes = sqlContext.sql("select Reported_by, Crime_type, Month, \
substring(Month, 0, 4) as Year,\
count(*) as Count from tbl_UKCrimereport group by Reported_by, Month, Crime_type")

In [ ]:
CrimeTypes.show()

In [ ]:
Crime_Types = CrimeTypes.withColumn("Year", year(col("Month"))).withColumn("MonthNum", month(col("Month")))

In [ ]:
Crime_Types.show()

In [ ]:
# Filter for only records for 2022

Crime_Types_2022 = Crime_Types.filter(Crime_Types["Year"] == "2022")

In [ ]:
Crime_Types_2022.show()

In [ ]:
# Group by Crime_type and count occurrences for 2022

Crime_Types_for_2022 = Crime_Types_2022.groupBy("Reported_by", "MonthNum", "Crime_type").sum("Count") \
.withColumnRenamed("sum(Count)", "Total_Count").orderBy(desc("Total_Count"))

In [ ]:
Crime_Types_for_2022.show()

In [ ]:
#Creating temp table for visualisation of  UK Counties Crime Type in 2022

Crime_Types_for_2022.registerTempTable("UKCounties_2022CrimeType")
sqlContext.sql("select * from UKCounties_2022CrimeType").show()

# TITANIC REPORT

#  Load the Dataset

In [ ]:
Titanic_data = pd.read_csv('/kaggle/input/titani-report/TitanicData1 (1).csv')

# Step 1: Explore and Visualize

In [ ]:
print(Titanic_data.shape)

In [ ]:
print(Titanic_data.head(10))
print(Titanic_data.info())

In [ ]:
Titanic_data.isnull().sum()

In [ ]:
#Visualize survival rates

import seaborn as sns
import matplotlib.pyplot as plt

# Survival by gender
sns.countplot(data=Titanic_data, x='Survived', hue='Sex')
plt.title('Survival by Gender')
plt.xlabel('Survival Status (0 = Did Not Survive, 1 = Survived)')
plt.ylabel('Count')
plt.legend(title='Gender', labels=['Male', 'Female'])
plt.show()

# Survival by class
sns.countplot(data=Titanic_data, x='Survived', hue='Pclass')
plt.title('Survival by Passenger Class')
plt.xlabel('Survival Status (0 = Did Not Survive, 1 = Survived)')
plt.ylabel('Count')
plt.legend(title='Passenger Class', labels=['1st Class', '2nd Class', '3rd Class'])
plt.show()

# Step 2: Feature Extraction and Preprocessing

In [ ]:
 #convert ‘Sex’ data using the rule: male1, female0
Titanic_data['Sex'] = Titanic_data['Sex'].map( {'male':1,'female':0} )

In [ ]:
#convert ‘Embarked’ data using the rule: Q2, S1, C0
Titanic_data['Embarked'] = Titanic_data['Embarked'].map( {'Q':2,'S':1,'C':0} )

In [ ]:
#fill in the missing value in ‘Embarked’ with mean value of Embarked
Titanic_data['Embarked'] = Titanic_data['Embarked'].fillna(Titanic_data['Embarked'].mean())


In [ ]:
# Fill missing values for 'Age' with median
Titanic_data['Age'].fillna(Titanic_data['Age'].mean(), inplace=True)

In [ ]:
# Fill missing values for 'Cabin' with 'Unknown'
Titanic_data['Cabin'] = Titanic_data['Cabin'].fillna('Unknown')

In [ ]:
print(Titanic_data.info())
Titanic_data.isnull().sum()

In [ ]:
New_Titanic_data= Titanic_data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

In [ ]:
print(type(New_Titanic_data))

In [ ]:
print(New_Titanic_data.dtypes)

In [ ]:
# import libraries
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [ ]:
from pyspark.ml import Pipeline

# Define stages of the pipeline
label_indexer = StringIndexer(inputCol="Survived", outputCol="label")
sex_indexer = StringIndexer(inputCol="Sex", outputCol="Sex_Index")
embarked_indexer = StringIndexer(inputCol="Embarked", outputCol="Embarked_Index")

# Define the VectorAssembler
assembler = VectorAssembler(inputCols=["Pclass", "Sex_Index", "Age", "SibSp", "Parch", "Fare", "Embarked_Index"], 
                            outputCol="features")

# Build the pipeline with all stages
Titanic_data_prep_pipe = Pipeline(stages=[label_indexer, sex_indexer, embarked_indexer, assembler])

Convert Titanic Data from pandas to pyspark

In [ ]:
print(type(New_Titanic_data))

In [ ]:
from pyspark.sql import types as T

# Define a schema for the PySpark DataFrame if necessary
schema = T.StructType([
    T.StructField("Pclass", T.IntegerType(), True),
    T.StructField("Sex", T.IntegerType(), True),
    T.StructField("Age", T.FloatType(), True),
    T.StructField("SibSp", T.IntegerType(), True),
    T.StructField("Parch", T.IntegerType(), True),
    T.StructField("Fare", T.FloatType(), True),
    T.StructField("Embarked", T.FloatType(), True),
    T.StructField("Survived", T.IntegerType(), True)
])

# Create the PySpark DataFrame with schema
NewPi_Titanic_data = spark.createDataFrame(New_Titanic_data, schema)

In [ ]:
print(type(NewPi_Titanic_data))

In [ ]:
# Fit the pipeline to the dataset
model = Titanic_data_prep_pipe.fit(NewPi_Titanic_data)

# Apply transformations to the data
cleaned_Titanic_data = model.transform(NewPi_Titanic_data)

# Show the transformed data
cleaned_Titanic_data.show()

In [ ]:
CleanT_data = cleaned_Titanic_data.select(['label','features'])
CleanT_data.show()

# Step 3: Build a Machine Learning Model

Training and Evaluation

In [ ]:
# Split the data into training and test sets (70/30%)
(training,testing) = CleanT_data.randomSplit([0.7,0.3])

# Naive Bayes Model

In [ ]:
# import machine learning method
from pyspark.ml.classification import NaiveBayes
NB = NaiveBayes()
# Train Naive bayes model using training data
Titanic_predictor = NB.fit(training)

In [ ]:
# Test the model on testing data
Titanic_test_results = Titanic_predictor.transform(testing)
Titanic_test_results.show()

In [ ]:
#  use the  MulticlassClassificationEvaluator function to extract more meaning out of the results.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
Titanic_eval = MulticlassClassificationEvaluator()
Titan = Titanic_eval.evaluate(Titanic_test_results)
print("Accuracy of Naive Bayes model at predicting Titanic: {}".format(Titan))

In [ ]:
# extract prediction and ground truth label from Titanic_test_results
Titanic_Label_Prediction = Titanic_test_results['label','prediction']
Titanic_Label_Prediction.show()

In [ ]:
# import multiclassmetrics for precision, recall,accuracy and confusion matrix calculation
from pyspark.mllib.evaluation import MulticlassMetrics
import matplotlib.pyplot as plt
import seaborn as sns
multiclass_metrics = MulticlassMetrics(Titanic_Label_Prediction.rdd)
precision_score = multiclass_metrics.weightedPrecision
recall_score = multiclass_metrics.weightedRecall
accuracy_score = multiclass_metrics.accuracy
print("Recall = {}".format(recall_score))
print("Precision = {}".format(precision_score))
print("Accuracy = {}".format(accuracy_score))
confusion_matrix = multiclass_metrics.confusionMatrix().toArray()
multiclass_metrics.confusionMatrix().toArray()
# Visualization using Seaborn and Matplotlib
plt.figure(figsize=(20, 5))
sns.heatmap(confusion_matrix, annot=True, fmt="g", cmap="Blues", xticklabels=True, yticklabels=True)
plt.title("Confusion Matrix for Naive Bayes Classifier")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.show()

In [ ]:
multiclass_metrics.confusionMatrix().toArray()

# Logistic Regression Model

In [ ]:
# import machine learning method logistic regression for classification
from pyspark.ml.classification import LogisticRegression
Logistic_Regression = LogisticRegression()
# Train Naive bayes model using training data
Titanic_predictor_lr = Logistic_Regression.fit(training)

In [ ]:
# Test the logistic regression model on testing data
Titanic_test_results_lr = Titanic_predictor_lr.transform(testing)
Titanic_test_results_lr.show()

In [ ]:
#  use the  MulticlassClassificationEvaluator function to extract more meaning out of the results.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
Titanic_eval_lr = MulticlassClassificationEvaluator()
Titanic_lr = Titanic_eval_lr.evaluate(Titanic_test_results_lr)
print("Accuracy of model at predicting Titanic with Logistic Regression is: {}".format(Titanic_lr))

In [ ]:
# extract prediction and ground truth label from Titanic_test_results
Titanic_Label_Prediction_lr = Titanic_test_results_lr['label','prediction']
Titanic_Label_Prediction_lr.show()

In [ ]:
# import multiclassmetrics for precision, recall,accuracy and confusion matrix calculation
from pyspark.mllib.evaluation import MulticlassMetrics
import matplotlib.pyplot as plt
import seaborn as sns
multiclass_metrics_lr = MulticlassMetrics(Titanic_Label_Prediction_lr.rdd)
precision_score_lr = multiclass_metrics_lr.weightedPrecision
recall_score_lr = multiclass_metrics_lr.weightedRecall
accuracy_score_lr = multiclass_metrics_lr.accuracy
print("Recall = {}".format(recall_score_lr))
print("Precision = {}".format(precision_score_lr))
print("Accuracy = {}".format(accuracy_score_lr))
confusion_matrix_lr = multiclass_metrics_lr.confusionMatrix().toArray()
multiclass_metrics_lr.confusionMatrix().toArray()
# Visualization using Seaborn and Matplotlib
plt.figure(figsize=(20, 5))
sns.heatmap(confusion_matrix_lr, annot=True, fmt="g", cmap="Blues", xticklabels=True, yticklabels=True)
plt.title("Confusion Matrix for Logistic Regression Classifier")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.show()

In [ ]:
multiclass_metrics_lr.confusionMatrix().toArray()

# Decision Tree Model

In [ ]:
# import machine learning method decision tree for classification
from pyspark.ml.classification import DecisionTreeClassifier
Decision_Tree = DecisionTreeClassifier()
# Decision Tree bayes model using training data
Titanic_predictor_dt = Decision_Tree.fit(training)

In [ ]:
# Test the Decision Tree model on testing data
Titanic_test_results_dt = Titanic_predictor_dt.transform(testing)
Titanic_test_results_dt.show()

In [ ]:
#  use the  MulticlassClassificationEvaluator function to extract more meaning out of the results.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
Titanic_eval_dt = MulticlassClassificationEvaluator()
Titanic_dt = Titanic_eval_dt.evaluate(Titanic_test_results)
print("Accuracy of model at predicting Titanic with Decision Tree: {}".format(Titanic_dt))

In [ ]:
# extract prediction and ground truth label from Titanic_test_results
Titanic_Label_Prediction_dt = Titanic_test_results_dt['label','prediction']
Titanic_Label_Prediction_dt.show()

In [ ]:
# import multiclassmetrics for precision, recall,accuracy and confusion matrix calculation
from pyspark.mllib.evaluation import MulticlassMetrics
import matplotlib.pyplot as plt
import seaborn as sns
multiclass_metrics_dt = MulticlassMetrics(Titanic_Label_Prediction_dt.rdd)
precision_score_dt = multiclass_metrics_dt.weightedPrecision
recall_score_dt = multiclass_metrics_dt.weightedRecall
accuracy_score_dt = multiclass_metrics_dt.accuracy
print("Recall = {}".format(recall_score_dt))
print("Precision = {}".format(precision_score_dt))
print("Accuracy = {}".format(accuracy_score_dt))
confusion_matrix_dt = multiclass_metrics_dt.confusionMatrix().toArray()
multiclass_metrics_dt.confusionMatrix().toArray()
# Visualization using Seaborn and Matplotlib
plt.figure(figsize=(20, 5))
sns.heatmap(confusion_matrix_dt, annot=True, fmt="g", cmap="Blues", xticklabels=True, yticklabels=True)
plt.title("Confusion Matrix for Decision Tree Classifier")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.show()

In [ ]:
multiclass_metrics_dt.confusionMatrix().toArray()

In [ ]:
EV_Countries = pd.read_csv('/kaggle/input/av-adopted-countries-dataset/AV Adopted Countries Dataset.csv')

In [ ]:
#West African Countries to be predicted

EV_African_Countries = pd.read_csv('/kaggle/input/ev-west-african-countries-dataset/EV_West_African_Countries_Data.csv')

In [ ]:
EV_Countries

In [ ]:
EV_African_Countries

In [ ]:
print(EV_Countries.shape)
print(EV_Countries.info())

In [ ]:
print(EV_African_Countries.shape)
print(EV_African_Countries.info())

**TRANSFORM DATASET**

In [ ]:
# Convert binary categorical values
EV_Countries['EV_Policy_Exists'] = EV_Countries['EV_Policy_Exists'].map({'yes': 1.0, 'no': 0.0})
EV_Countries['Tax_Incentives'] = EV_Countries['Tax_Incentives'].map({'yes': 1.0, 'no': 0.0})

In [ ]:
# Convert binary categorical values in the West African Dataset
EV_African_Countries['EV_Policy_Exists'] = EV_African_Countries['EV_Policy_Exists'].map({'yes': 1.0, 'no': 0.0})
EV_African_Countries['Tax_Incentives'] = EV_African_Countries['Tax_Incentives'].map({'yes': 1.0, 'no': 0.0})

In [ ]:
EV_Countries

In [ ]:
EV_African_Countries

In [ ]:
print(EV_Countries.shape)
print(EV_Countries.info())

In [ ]:
print(EV_African_Countries.shape)
print(EV_African_Countries.info())

**VISUALISE THE DATA SET**

In [ ]:
# Figure to see the correlation between features

import seaborn as sns

fig = plt.figure(figsize=(12,12))
Numerical_EV_Countries = EV_Countries.select_dtypes(include=np.number)  # select only numerical columns
sns.heatmap(Numerical_EV_Countries.corr(), vmax=1, square=True, annot=True)

#Brief idea about the correlation between features

In [ ]:
# Sort data by EV Penetration (%)
EV_Countries_sorted = EV_Countries.sort_values('EV Penetration (%)', ascending=False)

plt.figure(figsize=(14, 8))
sns.barplot(data=EV_Countries_sorted, x='EV Penetration (%)', y='Country', palette='viridis')

plt.title('EV Penetration (%) by Country')
plt.xlabel('EV Penetration (%)')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

In [ ]:
# Sort data by Renewable Energy Share
EV_Countries_sorted = EV_Countries.sort_values('Renewable_Share_percent', ascending=False)

plt.figure(figsize=(14, 8))
sns.barplot(data=EV_Countries_sorted, x='Renewable_Share_percent', y='Country', palette='viridis')

plt.title('Renewable Energy Share by Country')
plt.xlabel('Renewable Share (%)')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

In [ ]:
# Sort countries by awareness score (descending)
EV_Countries_sorted = EV_Countries.sort_values('EV_Public_Awareness_Score', ascending=False)

plt.figure(figsize=(14, 8))
sns.barplot(data=EV_Countries_sorted, x='EV_Public_Awareness_Score', y='Country', palette='coolwarm')

plt.title('EV Public Awareness Score by Country')
plt.xlabel('Public Awareness Score')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

In [ ]:
EV_Countries.loc[EV_Countries['EV_Public_Awareness_Score'].idxmax(), 'Country']

In [ ]:
# Sort data by Renewable Energy Share for Dataset 2
EV_African_Countries_sorted = EV_African_Countries.sort_values('Renewable_Share_percent', ascending=False)

plt.figure(figsize=(14, 8))
sns.barplot(data=EV_African_Countries_sorted, x='Renewable_Share_percent', y='Country', palette='viridis')

plt.title('Renewable Energy Share by Country')
plt.xlabel('Renewable Share (%)')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

In [ ]:
# Sort countries by awareness score (descending) for Dataset 2
EV_African_Countries_sorted = EV_African_Countries.sort_values('EV_Public_Awareness_Score', ascending=False)

plt.figure(figsize=(14, 8))
sns.barplot(data=EV_African_Countries_sorted, x='EV_Public_Awareness_Score', y='Country', palette='coolwarm')

plt.title('EV Public Awareness Score by Country')
plt.xlabel('Public Awareness Score')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

In [ ]:
# Sort countries by GDP (descending) for Dataset 2
EV_African_Countries_sorted = EV_African_Countries.sort_values('GDP_per_Capita_USD', ascending=False)

plt.figure(figsize=(14, 8))
sns.barplot(data=EV_African_Countries_sorted, x='GDP_per_Capita_USD', y='Country', palette='coolwarm')

plt.title('GDP_per_Capita_USD by Country')
plt.xlabel('GDP_per_Capita_USD')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

In [ ]:
# Sort countries by GDP (descending) for Dataset 2
EV_African_Countries_sorted = EV_African_Countries.sort_values('Social_Media_Sentiment', ascending=False)

plt.figure(figsize=(14, 8))
sns.barplot(data=EV_African_Countries_sorted, x='Social_Media_Sentiment', y='Country', palette='coolwarm')

plt.title('Social_Media_Sentiment by Country')
plt.xlabel('Social_Media_Sentiment')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

In [ ]:
# Sort countries by ICT_Readiness_Index (descending) for Dataset 2
EV_African_Countries_sorted = EV_African_Countries.sort_values('ICT_Readiness_Index', ascending=False)

plt.figure(figsize=(14, 8))
sns.barplot(data=EV_African_Countries_sorted, x='ICT_Readiness_Index', y='Country', palette='coolwarm')

plt.title('ICT_Readiness_Index by Country')
plt.xlabel('ICT_Readiness_Index')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

**FEATURE EXTRACTION AND PREPROCESSING**

In [ ]:
#Creating new category column to classify the target variable into low medium and low

bins = [0, 1, 5, np.inf]
labels = ['Low', 'Medium', 'High']

# Create new category column
EV_Countries['Penetration_Category'] = pd.cut(EV_Countries['EV Penetration (%)'], bins=bins, labels=labels, right=False)

# Display the result
print(EV_Countries[['EV Penetration (%)', 'Penetration_Category']])

In [ ]:
#Converting the new column 'Penetration_Category' data using the rule: High 2, Medium 1, Low 0
#This is because models require numeric targets to perform learning operations

EV_Countries['Penetration_Category'] = EV_Countries['Penetration_Category'].map( {'High':2,'Medium':1,'Low':0} )

In [ ]:
EV_Countries

In [ ]:
#Creating new category column to classify the target variable into low medium and low

bins = [0, 1, 5, np.inf]
labels = ['Low', 'Medium', 'High']

# Create new category column
EV_African_Countries['Penetration_Category'] = pd.cut(EV_African_Countries['EV Penetration (%)'], bins=bins, labels=labels, right=False)

# Display the result
print(EV_African_Countries[['EV Penetration (%)', 'Penetration_Category']])

In [ ]:
#Converting the new column 'Penetration_Category' data using the rule: High 2, Medium 1, Low 0
#This is because models require numeric targets to perform learning operations

EV_African_Countries['Penetration_Category'] = EV_African_Countries['Penetration_Category'].map( {'High':2,'Medium':1,'Low':0} )

In [ ]:
EV_African_Countries

In [ ]:
#Drop Country because it is an identifier (name), not useful for prediction
#Drop EV Penetration (%), better to predict with Penetration_Category

New_EV_Countries= EV_Countries.drop(columns=['Country', 'EV Penetration (%)'])

In [ ]:
New_EV_Countries

In [ ]:
#Drop Country in the  because it is an identifier (name), not useful for prediction

New_EV_African_Countries= EV_African_Countries.drop(columns=['Country', 'EV Penetration (%)'])

In [ ]:
New_EV_African_Countries

In [ ]:
New_EV_Countries = New_EV_Countries.rename(columns={
    'Total Vehicles est.': 'Total_Vehicles_est',
    'Registered EVs est.': 'Registered_EVs_est',
    'EV Policy Exists': 'EV_Policy_Exists',
    'Tax Incentives': 'Tax_Incentives',
    'EV Public Awareness Score': 'EV_Public_Awareness_Score',
    'Social Media Sentiment': 'Social_Media_Sentiment',
    'ICT Readiness Index': 'ICT_Readiness_Index'
})

New_EV_Countries

In [ ]:
New_EV_African_Countries = New_EV_African_Countries.rename(columns={
    'Total Vehicles est.': 'Total_Vehicles_est',
    'Registered EVs est.': 'Registered_EVs_est',
    'EV Policy Exists': 'EV_Policy_Exists',
    'Tax Incentives': 'Tax_Incentives',
    'EV Public Awareness Score': 'EV_Public_Awareness_Score',
    'Social Media Sentiment': 'Social_Media_Sentiment',
    'ICT Readiness Index': 'ICT_Readiness_Index'
})

New_EV_African_Countries

In [ ]:
# import libraries
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.ml import Pipeline

In [ ]:
# Define stages of the pipeline
# Index target column (Penetration_Category)

label_indexer = StringIndexer(inputCol="Penetration_Category", outputCol="label")

#Choosing features

feature_cols = [
    "GDP_per_Capita_USD",
    "Inflation_Rate_percent",
    "Population_millions",
    "Urbanization_Rate_percent",
    "Unemployment_Rate_percent",
    "Electricity_Access_percent",
    "Renewable_Share_percent",
    "EV_Charging_Stations",
    "Fuel_Price_USD_per_litre",
    "EV_Policy_Exists",
    "Tax_Incentives",
    "EV_Public_Awareness_Score",
    "Social_Media_Sentiment",
    "ICT_Readiness_Index",
    "Total_Vehicles_est",
    "Registered_EVs_est"
]

#Assemble the feature vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Build the pipeline with all stages
EV_Countries_data_prep_pipe = Pipeline(stages=[label_indexer, assembler])

In [ ]:
# Index target column (Penetration_Category)

label_indexer = StringIndexer(inputCol="Penetration_Category", outputCol="label")

#Choosing features
feature_cols = [
    "GDP_per_Capita_USD",
    "Inflation_Rate_percent",
    "Population_millions",
    "Urbanization_Rate_percent",
    "Unemployment_Rate_percent",
    "Electricity_Access_percent",
    "Renewable_Share_percent",
    "EV_Charging_Stations",
    "Fuel_Price_USD_per_litre",
    "EV_Policy_Exists",
    "Tax_Incentives",
    "EV_Public_Awareness_Score",
    "Social_Media_Sentiment",
    "ICT_Readiness_Index",
    "Total_Vehicles_est",
    "Registered_EVs_est"
]

#Assemble the feature vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Build the pipeline with all stages
EV_African_Countries_data_prep_pipe = Pipeline(stages=[label_indexer, assembler])

In [ ]:
print(type(New_EV_Countries))

In [ ]:
print(type(New_EV_African_Countries))

In [ ]:
#Convert New_EV_Countries Data from pandas to pyspark

from pyspark.sql import types as T

# Define a schema for the PySpark DataFrame if necessary
schema = T.StructType([
    T.StructField("GDP_per_Capita_USD",T.IntegerType(), True),
    T.StructField("Inflation_Rate_percent", T.FloatType(), True),
    T.StructField("Population_millions", T.FloatType(), True),
    T.StructField("Urbanization_Rate_percent", T.FloatType(), True),
    T.StructField("Unemployment_Rate_percent", T.FloatType(), True),
    T.StructField("Electricity_Access_percent", T.IntegerType(), True),
    T.StructField("Renewable_Share_percent", T.FloatType(), True),
    T.StructField("EV_Charging_Stations", T.IntegerType(), True),
    T.StructField("Fuel_Price_USD_per_litre", T.FloatType(), True),
    T.StructField("EV_Policy_Exists", T.FloatType(), True),
    T.StructField("Tax_Incentives", T.FloatType(), True),
    T.StructField("EV_Public_Awareness_Score", T.FloatType(), True),
    T.StructField("Social_Media_Sentiment", T.FloatType(), True),
    T.StructField("ICT_Readiness_Index", T.FloatType(), True),
    T.StructField("Total_Vehicles_est", T.IntegerType(), True),
    T.StructField("Registered_EVs_est", T.IntegerType(), True),
    T.StructField("Penetration_Category", T.IntegerType(), True)
])

# Create the PySpark DataFrame with schema for Dataset 1
Pi_New_EV_Countries = spark.createDataFrame(New_EV_Countries, schema)

print(type(Pi_New_EV_Countries))

In [ ]:
#Convert New_EV_African_Countries Data, Dataset 2 from pandas to pyspark
# Define a schema for the PySpark DataFrame if necessary
schema2 = T.StructType([
    T.StructField("GDP_per_Capita_USD",T.IntegerType(), True),
    T.StructField("Inflation_Rate_percent", T.FloatType(), True),
    T.StructField("Population_millions", T.FloatType(), True),
    T.StructField("Urbanization_Rate_percent", T.FloatType(), True),
    T.StructField("Unemployment_Rate_percent", T.FloatType(), True),
    T.StructField("Electricity_Access_percent", T.FloatType(), True),
    T.StructField("Renewable_Share_percent", T.FloatType(), True),
    T.StructField("EV_Charging_Stations", T.IntegerType(), True),
    T.StructField("Fuel_Price_USD_per_litre", T.FloatType(), True),
    T.StructField("EV_Policy_Exists", T.FloatType(), True),
    T.StructField("Tax_Incentives", T.FloatType(), True),
    T.StructField("EV_Public_Awareness_Score", T.FloatType(), True),
    T.StructField("Social_Media_Sentiment", T.FloatType(), True),
    T.StructField("ICT_Readiness_Index", T.FloatType(), True),
    T.StructField("Total_Vehicles_est", T.IntegerType(), True),
    T.StructField("Registered_EVs_est", T.IntegerType(), True),
    T.StructField("Penetration_Category", T.IntegerType(), True)
])

# Create the PySpark DataFrame with schema
Pi_New_EV_African_Countries = spark.createDataFrame(New_EV_African_Countries, schema2)

print(type(Pi_New_EV_African_Countries))

In [ ]:
Pi_New_EV_Countries.show()

In [ ]:
Pi_New_EV_African_Countries.show()

In [ ]:
# Fit the pipeline to the dataset
model = EV_Countries_data_prep_pipe.fit(Pi_New_EV_Countries)

# Apply transformations to the data
cleaned_EV_Countries = model.transform(Pi_New_EV_Countries)

# Show the transformed data
cleaned_EV_Countries.show()

In [ ]:
# Fit the pipeline to the dataset
model2 = EV_African_Countries_data_prep_pipe.fit(Pi_New_EV_African_Countries)

# Apply transformations to the data
cleaned_WA_Countries = model2.transform(Pi_New_EV_African_Countries)

# Show the transformed data
cleaned_WA_Countries.show()

In [ ]:
CleanEV_dataset = cleaned_EV_Countries.select(['label','features'])
CleanEV_dataset.show()

In [ ]:
CleanEV_WA_dataset = cleaned_WA_Countries.select(['label','features'])
CleanEV_WA_dataset.show()

**Build a Machine Learning Model**

In [ ]:
# Training and Evaluation
# Trying one model on Dataset 1 to see how it turns out 
# Split Dataset 1 into training and test sets (70/30%)
(training,testing) = CleanEV_dataset.randomSplit([0.7,0.3])

In [ ]:
# import machine learning method
from pyspark.ml.classification import NaiveBayes
NB = NaiveBayes()
# Train Naive bayes model using training data
EV_Adoption_predictor = NB.fit(training)

In [ ]:
# Test the model on testing data of Dataset 1
EV_test_results = EV_Adoption_predictor.transform(testing)
EV_test_results.show()

In [ ]:
# Test the model on West African Dataset, Dataset 2
EV_WA_results = EV_Adoption_predictor.transform(CleanEV_WA_dataset)
EV_WA_results.show()

In [ ]:
#  use the  MulticlassClassificationEvaluator function to extract more meaning out of the results.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
EV_eval = MulticlassClassificationEvaluator()
EV_Naive_MCE = EV_eval.evaluate(EV_WA_results)
print("Accuracy of Naive Bayes model at predicting Electric Vehicle: {}".format(EV_Naive_MCE))

In [ ]:
# extract prediction and ground truth label from Titanic_test_results
EV_WA_Prediction = EV_WA_results['label','prediction']
EV_WA_Prediction.show()

In [ ]:
# import multiclassmetrics for precision, recall,accuracy and confusion matrix calculation
from pyspark.mllib.evaluation import MulticlassMetrics
import matplotlib.pyplot as plt
import seaborn as sns
multiclass_metrics = MulticlassMetrics(EV_WA_Prediction.rdd)
precision_score = multiclass_metrics.weightedPrecision
recall_score = multiclass_metrics.weightedRecall
accuracy_score = multiclass_metrics.accuracy
f1_score = multiclass_metrics.weightedFMeasure()
print("Recall = {}".format(recall_score))
print("Precision = {}".format(precision_score))
print("Accuracy = {}".format(accuracy_score))
print("F1_Score = {}".format(f1_score))
confusion_matrix = multiclass_metrics.confusionMatrix().toArray()
multiclass_metrics.confusionMatrix().toArray()
# Visualization using Seaborn and Matplotlib
plt.figure(figsize=(20, 5))
sns.heatmap(confusion_matrix, annot=True, fmt="g", cmap="Blues", xticklabels=True, yticklabels=True)
plt.title("Confusion Matrix for Naive Bayes Classifier")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.show()

In [ ]:
multiclass_metrics.confusionMatrix().toArray()

**Logistic Regression Model**

In [ ]:
# import machine learning method logistic regression for classification
from pyspark.ml.classification import LogisticRegression
Logistic_Regression = LogisticRegression()
# Train Logistic Regression model using training data
EV_Adoption_LR = Logistic_Regression.fit(training)

In [ ]:
# Test the logistic regression model on Dataset 2
EV_WA_results_LR = EV_Adoption_LR.transform(CleanEV_WA_dataset)
EV_WA_results_LR.show()

In [ ]:
#  use the  MulticlassClassificationEvaluator function to extract more meaning out of the results.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
EV_WA_eval_LR = MulticlassClassificationEvaluator()
EV_WA_LR = EV_WA_eval_LR.evaluate(EV_WA_results_LR)
print("Accuracy of model at predicting Electric Vehicle with Logistic Regression is: {}".format(EV_WA_LR))

In [ ]:
# extract prediction and ground truth label from EV_WA_results_LR
EV_WA_Label_Prediction_LR = EV_WA_results_LR['label','prediction']
EV_WA_Label_Prediction_LR.show()

In [ ]:
# import multiclassmetrics for precision, recall,accuracy and confusion matrix calculation
from pyspark.mllib.evaluation import MulticlassMetrics
import matplotlib.pyplot as plt
import seaborn as sns
multiclass_metrics_lr = MulticlassMetrics(EV_WA_Label_Prediction_LR.rdd)
precision_score_lr = multiclass_metrics_lr.weightedPrecision
recall_score_lr = multiclass_metrics_lr.weightedRecall
accuracy_score_lr = multiclass_metrics_lr.accuracy
f1_score_lr = multiclass_metrics_lr.weightedFMeasure()
print("Recall = {}".format(recall_score_lr))
print("Precision = {}".format(precision_score_lr))
print("Accuracy = {}".format(accuracy_score_lr))
print("F1_Score = {}".format(f1_score_lr))
confusion_matrix_lr = multiclass_metrics_lr.confusionMatrix().toArray()
multiclass_metrics_lr.confusionMatrix().toArray()
# Visualization using Seaborn and Matplotlib
plt.figure(figsize=(20, 5))
sns.heatmap(confusion_matrix_lr, annot=True, fmt="g", cmap="Blues", xticklabels=True, yticklabels=True)
plt.title("Confusion Matrix for Logistic Regression Classifier")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.show()

In [ ]:
multiclass_metrics_lr.confusionMatrix().toArray()

**Decision Tree Model**

In [ ]:
# import machine learning method decision tree for classification
from pyspark.ml.classification import DecisionTreeClassifier
Decision_Tree = DecisionTreeClassifier()
# Decision Tree bayes model using training data
EV_Adoption_DT = Decision_Tree.fit(training)

In [ ]:
# Test the Decision Tree model on Dataset 2
EV_WA_results_DT = EV_Adoption_DT.transform(CleanEV_WA_dataset)
EV_WA_results_DT.show()

In [ ]:
#  use the  MulticlassClassificationEvaluator function to extract more meaning out of the results.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
EV_WA_eval_DT = MulticlassClassificationEvaluator()
EV_WA_DT = EV_WA_eval_DT.evaluate(EV_WA_results_DT)
print("Accuracy of model at predicting Electric Vehicle with Logistic Regression is: {}".format(EV_WA_LR))

In [ ]:
# extract prediction and ground truth label from EV_WA_results_DT
EV_WA_Label_Prediction_DT = EV_WA_results_DT['label','prediction']
EV_WA_Label_Prediction_DT.show()

In [ ]:
# import multiclassmetrics for precision, recall,accuracy and confusion matrix calculation
from pyspark.mllib.evaluation import MulticlassMetrics
import matplotlib.pyplot as plt
import seaborn as sns
multiclass_metrics_dt = MulticlassMetrics(EV_WA_Label_Prediction_DT.rdd)
precision_score_dt = multiclass_metrics_dt.weightedPrecision
recall_score_dt = multiclass_metrics_dt.weightedRecall
accuracy_score_dt = multiclass_metrics_dt.accuracy
f1_score_dt = multiclass_metrics_dt.weightedFMeasure()
print("Recall = {}".format(recall_score_dt))
print("Precision = {}".format(precision_score_dt))
print("Accuracy = {}".format(accuracy_score_dt))
print("F1_Score = {}".format(f1_score_dt))
confusion_matrix_dt = multiclass_metrics_dt.confusionMatrix().toArray()
multiclass_metrics_dt.confusionMatrix().toArray()
# Visualization using Seaborn and Matplotlib
plt.figure(figsize=(20, 5))
sns.heatmap(confusion_matrix_dt, annot=True, fmt="g", cmap="Blues", xticklabels=True, yticklabels=True)
plt.title("Confusion Matrix for Decision Tree Classifier")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.show()

In [ ]:
multiclass_metrics_dt.confusionMatrix().toArray()

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

RF = RandomForestClassifier()
EV_Adoption_RF = RF.fit(training)

In [ ]:
# Test the Random Forest model on Dataset 2
EV_WA_results_RF = EV_Adoption_RF.transform(CleanEV_WA_dataset)
EV_WA_results_RF.show()

In [ ]:
#  use the  MulticlassClassificationEvaluator function to extract more meaning out of the results.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
EV_WA_eval_RF = MulticlassClassificationEvaluator()
EV_WA_RF = EV_WA_eval_RF.evaluate(EV_WA_results_RF)
print("Accuracy of model at predicting Electric Vehicle with Random Forest is: {}".format(EV_WA_RF))

In [ ]:
# extract prediction and ground truth label from EV_WA_results_DT
EV_WA_Label_Prediction_RF = EV_WA_results_RF['label','prediction']
EV_WA_Label_Prediction_RF.show()

In [ ]:
# import multiclassmetrics for precision, recall,accuracy and confusion matrix calculation
from pyspark.mllib.evaluation import MulticlassMetrics
import matplotlib.pyplot as plt
import seaborn as sns
multiclass_metrics_rf = MulticlassMetrics(EV_WA_Label_Prediction_RF.rdd)
precision_score_rf = multiclass_metrics_rf.weightedPrecision
recall_score_rf = multiclass_metrics_rf.weightedRecall
accuracy_score_rf = multiclass_metrics_rf.accuracy
f1_score_rf = multiclass_metrics_rf.weightedFMeasure()
print("Recall = {}".format(recall_score_rf))
print("Precision = {}".format(precision_score_rf))
print("Accuracy = {}".format(accuracy_score_rf))
print("F1_Score = {}".format(f1_score_rf))
confusion_matrix_rf = multiclass_metrics_rf.confusionMatrix().toArray()
multiclass_metrics_rf.confusionMatrix().toArray()
# Visualization using Seaborn and Matplotlib
plt.figure(figsize=(20, 5))
sns.heatmap(confusion_matrix_rf, annot=True, fmt="g", cmap="Blues", xticklabels=True, yticklabels=True)
plt.title("Confusion Matrix for Random Forest Classifier")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.show()

In [ ]:
multiclass_metrics_rf.confusionMatrix().toArray()

In [ ]:
from pyspark.ml.classification import GBTClassifier
# Initialize the model
GBT = GBTClassifier()
EV_Adoption_GBT = GBT.fit(training)

In [ ]:
# Test the  model on Dataset 2
EV_WA_results_GBT = EV_Adoption_GBT.transform(CleanEV_WA_dataset)
EV_WA_results_GBT.show()

In [ ]:
#  use the  MulticlassClassificationEvaluator function to extract more meaning out of the results.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
EV_WA_eval_GBT = MulticlassClassificationEvaluator()
EV_WA_GBT = EV_WA_eval_GBT.evaluate(EV_WA_results_GBT)
print("Accuracy of model at predicting Electric Vehicle with Gradient-Boosted Trees is: {}".format(EV_WA_GBT))

In [ ]:
# extract prediction and ground truth label from EV_WA_results_DT
EV_WA_Label_Prediction_GBT = EV_WA_results_GBT['label','prediction']
EV_WA_Label_Prediction_GBT.show()

In [ ]:
# import multiclassmetrics for precision, recall,accuracy and confusion matrix calculation
from pyspark.mllib.evaluation import MulticlassMetrics
import matplotlib.pyplot as plt
import seaborn as sns
multiclass_metrics_GBT = MulticlassMetrics(EV_WA_Label_Prediction_GBT.rdd)
precision_score_GBT = multiclass_metrics_GBT.weightedPrecision
recall_score_GBT = multiclass_metrics_GBT.weightedRecall
accuracy_score_GBT = multiclass_metrics_GBT.accuracy
f1_score_GBT = multiclass_metrics_GBT.weightedFMeasure()
print("Recall = {}".format(recall_score_GBT))
print("Precision = {}".format(precision_score_GBT))
print("Accuracy = {}".format(accuracy_score_GBT))
print("F1_Score = {}".format(f1_score_GBT))
confusion_matrix_GBT = multiclass_metrics_GBT.confusionMatrix().toArray()
multiclass_metrics_GBT.confusionMatrix().toArray()
# Visualization using Seaborn and Matplotlib
plt.figure(figsize=(20, 5))
sns.heatmap(confusion_matrix_GBT, annot=True, fmt="g", cmap="Blues", xticklabels=True, yticklabels=True)
plt.title("Confusion Matrix for Gradient-Boosted Trees Classifier")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.show()

In [ ]:
multiclass_metrics_GBT.confusionMatrix().toArray()

In [ ]:
#Linear Support Vector Machine (SVM) 
from pyspark.ml.classification import LinearSVC
SVM = LinearSVC()
EV_Adoption_SVM = SVM.fit(training)

In [ ]:
# Test the  model on Dataset 2
EV_WA_results_SVM = EV_Adoption_SVM.transform(CleanEV_WA_dataset)
EV_WA_results_SVM.show()

In [ ]:
#  use the  MulticlassClassificationEvaluator function to extract more meaning out of the results.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
EV_WA_eval_SVM = MulticlassClassificationEvaluator()
EV_WA_SVM = EV_WA_eval_SVM.evaluate(EV_WA_results_SVM)
print("Accuracy of model at predicting Electric Vehicle with Linear Support Vector Machine Classifier is: {}".format(EV_WA_GBT))

In [ ]:
# extract prediction and ground truth label from EV_WA_results_DT
EV_WA_Label_Prediction_SVM = EV_WA_results_SVM['label','prediction']
EV_WA_Label_Prediction_SVM.show()

In [ ]:
# import multiclassmetrics for precision, recall,accuracy and confusion matrix calculation
from pyspark.mllib.evaluation import MulticlassMetrics
import matplotlib.pyplot as plt
import seaborn as sns
multiclass_metrics_SVM = MulticlassMetrics(EV_WA_Label_Prediction_SVM.rdd)
precision_score_SVM = multiclass_metrics_SVM.weightedPrecision
recall_score_SVM = multiclass_metrics_SVM.weightedRecall
accuracy_score_SVM = multiclass_metrics_SVM.accuracy
f1_score_SVM = multiclass_metrics_SVM.weightedFMeasure()
print("Recall = {}".format(recall_score_SVM))
print("Precision = {}".format(precision_score_SVM))
print("Accuracy = {}".format(accuracy_score_SVM))
print("F1_Score = {}".format(f1_score_SVM))
confusion_matrix_SVM = multiclass_metrics_SVM.confusionMatrix().toArray()
multiclass_metrics_SVM.confusionMatrix().toArray()
# Visualization using Seaborn and Matplotlib
plt.figure(figsize=(20, 5))
sns.heatmap(confusion_matrix_SVM, annot=True, fmt="g", cmap="Blues", xticklabels=True, yticklabels=True)
plt.title("Confusion Matrix for Linear Support Vector Machine Classifier")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.show()

In [ ]:
multiclass_metrics_SVM.confusionMatrix().toArray()

In [ ]:
from tabulate import tabulate

# Sample metrics (replace with your actual scores)
data = [
    ["Logistic Regression", accuracy_score_lr, precision_score_lr, recall_score_lr, f1_score_lr],
    ["Random Forest Classifier", accuracy_score_rf, precision_score_rf, recall_score_rf, f1_score_rf],
    ["XGB Classifier", accuracy_score_GBT, precision_score_GBT, recall_score_GBT, f1_score_GBT],
    ["Support Vector Machine", accuracy_score_SVM, precision_score_SVM, recall_score_SVM, f1_score_SVM],
    ["Decision Tree Classifier", accuracy_score_dt, precision_score_dt, recall_score_dt, f1_score_dt],
    ["Gaussian Naive Bayes", accuracy_score, precision_score, recall_score, f1_score]
]

columns = ["Algorithm", "Accuracy Score", "Precision", "Recall", "F1 Score"]

# Print the tabulated result
print(tabulate(data, headers=columns, tablefmt="fancy_grid", floatfmt=".3f"))
